### Tools
Models can request to call tools that perform tasks such as fetching data from a database, searching the web, or running code. Tools are pairings of:
1. A schema, including the name of the tool, a description, and/or argument definitions (often a JSON schema)
2. A function or coroutine to execute.

In [ ]:
# Import the os module to work with environment variables
import os

# Import the function to initialize a chat-based LLM from LangChain
from langchain.chat_models import init_chat_model

# Load the Google API key from system environment variables
# Make sure GOOGLE_API_KEY is already set in your system
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

# Initialize the Gemini 2.5 Flash model using Google's Generative AI provider
# This model will be used for chat-based question answering
model = init_chat_model(
    "gemini-2.5-flash",
    model_provider="google_genai"
)

# Send a prompt (question) to the model
# The invoke() method sends the input and receives the AI-generated response
response = model.invoke("Types of Machine Learning Model?")

# Print or return the model's response
response


AIMessage(content='Machine learning models can be categorized in several ways, primarily by **how they learn (learning paradigm)**, **what task they perform (function)**, and their **underlying structure or algorithm family**.\n\nHere\'s a breakdown of the main types:\n\n---\n\n### 1. By Learning Paradigm (How They Learn)\n\nThis is the most fundamental classification.\n\n*   **Supervised Learning Models:**\n    *   **Concept:** Learn from labeled data, meaning the input data is paired with the correct output. The model learns a mapping from input to output.\n    *   **Goal:** Predict future outcomes for unseen data based on the learned patterns.\n    *   **Sub-types/Tasks:**\n        *   **Classification:** Predicts a categorical label (e.g., "spam" or "not spam", "cat" or "dog", "disease A", "disease B", "healthy").\n            *   *Examples:* Logistic Regression, Support Vector Machines (SVM), Decision Trees, Random Forests, K-Nearest Neighbors (k-NN), Neural Networks.\n        *  

### Enabling External Tools in LangChain

In [ ]:
from langchain.tools import tool

# 1. Define the Tool using a Decorator
# The @tool decorator converts a standard Python function into a LangChain Tool.
# The AI uses the function name, arguments, and the docstring ("Get the weather...") 
# to understand when and how to use this tool.
@tool
def get_weather(location: str) -> str:
    """Get the weather at a location"""
    return f"it's sunny in {location}"

# 2. Bind the Tool to the Model
# model.bind_tools() takes your list of functions and tells the Gemini model:
# "You now have permission to ask to run these specific functions."
model_with_tools = model.bind_tools([get_weather])

In [6]:
response = model_with_tools.invoke("What's the weather like in Boston?")
print(response)
for tool_call in response.tool_calls:
    # View tool calls made by the model
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")

content='' additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"location": "Boston"}'}, '__gemini_function_call_thought_signatures__': {'34e94a70-b7d3-44c6-b5e0-9f9f315d3c29': 'CukBAXLI2nx94XIs3sxi3bbsXlQuCZsXMXf+YEjzwI2nVBiB4LH0SMl/mXnaA0nzNkRCIREg+ouzx0JaeisrN5ETEmJxdua40aIq+kLQEsEwr4qwOO0LlBQWuCprxq3wWFI9PYKKqe17ZXlQaV4Tmk5pmUuHUThi9y2WUWJGNYKucrcZq/X1SQMY8uM0myHDGzSLKCNWymf+Paew6OO16dAym5mh8k+nBabHclyqG1hk8Me3EQYAGMxG/eUN32pNOLTLk+uxMB1yToqLohrQDBhurp7IX9t4b2bjuH2dtxE4DsPxMzdvD9qL7Fc='}} response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019be4d7-aaed-77b0-aac3-3937751fedc0-0' tool_calls=[{'name': 'get_weather', 'args': {'location': 'Boston'}, 'id': '34e94a70-b7d3-44c6-b5e0-9f9f315d3c29', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 48, 'output_tokens': 61, 'total_tokens': 109, 'input_token_details': {'cache_read': 0}, 'output_token_de

### Tool Execution Loops

In [7]:
# Step 1: Model generates tool calls
messages = [{"role": "user", "content": "What's the weather in Boston?"}]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

# Step 2: Execute tools and collect results
for tool_call in ai_msg.tool_calls:
    # Execute the tool with the generated arguments
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

# Step 3: Pass results back to model for final response
final_response = model_with_tools.invoke(messages)
print(final_response.text)
# "The current weather in Boston is 72°F and sunny."

It's sunny in Boston.


In [9]:
messages

[{'role': 'user', 'content': "What's the weather in Boston?"},
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"location": "Boston"}'}, '__gemini_function_call_thought_signatures__': {'b32b6cf0-5507-4615-8e24-c9da2e9e7a2c': 'CukBAXLI2nz5pko0VxBP5TQdxPbscPdKI6whjTj5FRcHUmLG6nb0VVfywlnZNVQmNhFdbd5JJuRcW7dDhQQ4z2bWNEmExs6M3c2G2ki1A+M9ert5gWfBOkDCT5TSWbhrv5+9jci218JA9iHEVazs2ctLPNSjt1BoEiNfqDQmVqEbVx+6Nl7bISUvT6fwi3b11jAbV6FCFe+sTtuxbCAGnjxLIy4YsD5Y0tdp/3ne3fOB0bYfcj7xwauOm/OW6IXYW6BJlk8cTB3aqQTxa8iUav7s3c91+MuFXtHtf2X7WVOIReHLDM587z6x60Q='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019be4d9-104f-77d3-a30e-1f6bc487f3cb-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Boston'}, 'id': 'b32b6cf0-5507-4615-8e24-c9da2e9e7a2c', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 47, 'output_tokens': 61,